# Scientific Machine Learning: PINN For Audio Processing





**ASSA 2025 Autumn school**

Mirco PEZZOLI - Politecnico di Milano, Milan, Italy

> Note: this notebook is based on my [Forum Acusticum 2025 summer school](https://github.com/m-pexx/fa25-summer-school) and the [EUSIPCO 2025 tutorial](https://github.com/Chutlhu/EUSIPCO25_PIML_tutorial) on PIML for audio processing
[Credits include Doc. [Diego Di Carlo](https://diegodicarlo.com/index.html) and Prof. [Shoichi Koyama](https://www.sh01.org/)]

## Tutorial Goals
- Implement a **Physics-Informed Neural Network (PINN)** for solving the foward problem of the *1D acoustic wave equation*



**This tutorial will NOT cover**
- Other "Physics-informed" DNN models (e.g., Neural Operator, etc)
- JAX framework for DNN in Python

Prof. Borrel-Jensen will cover them!!


# Let's start!

In [ ]:
# try writing and executing some code here

In [ ]:

# We start importing some libraries
import time
import random
import math
import sys
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile

sys.path.append('.')

In [ ]:
# If GPU (if running in Colab: Runtime -> Change runtime type -> GPU (recommended))
#!nvidia-smi  # Uncomment to see GPU

In [ ]:
# Run on the available device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Current device:', device)

# Set the seed number
def reset_seeds(seed=123):
    """Reset all random seeds to e nsure reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

SEED = 1234
reset_seeds(SEED) # <-- copy this before declaring any model, for reproducibility

All set with the necesarry settings. Let start! 🚀


## 1D Traveling Wave

The simplest wave is the (spatially) one-dimensional sine wave.


It is the solution of the wave equation PDE:

$$
\nabla^2_{z} U(t,z) - \frac{1}{c^2} \frac{\partial^2}{\partial t^2} U(t, z) = 0
$$

- for the domain $z\in \Omega = [0,L)$,
- **Initial conditions** (i.e. on how the wave is produced): for a pure traveling wave
    - $u(0,x) =\sin(kz)$ for the initial position
    - $u_t(0,x) = -\omega\,\cos(kz)$ for the inital velocity
- **boudary conditions** (i.e., what is happening on the boundary of the domain ): periodic, that is,
    - $u(t,0)=u(t,L)$
    - $u_z(t,0)=u_z(t,L)$

## Analytical solution

The above equation admits unique solution, known as **1D travelling wave** in the form of

$$
u(t,z) = \sin\big(kz - \omega t\big).
$$
where
- $k = \frac{2\pi n}{L}$ is the *wave number*
- $\omega = ck,$ is the *angular frequency*
- $n \in \mathbb{N}_+$ is the *mode index*

Note that both $k$ and $\omega$ are defined by the initial and boundary conditions

Let's code it! 💻

In [ ]:
# Physical / modal parameters
const = {}     # to store the simulation params
const['L'] = L = 1.0        # domain length
const['c'] = c = 1.0        # propagation speed
const['n'] = n = 1          # mode index (try 1,2,3,...)
const['T'] = T = 1.0        # max simulation time

k = 2*np.pi*n/L
omega = c*k

Nz = 200 # num of space points
Nt = 100 # num of time points
z  = np.linspace(0, L, Nz, endpoint=False)  # periodic grid
times  = np.linspace(0, T, Nt)

In [ ]:
def wave_1D_periodic(t, z, L=1, c=1, n=1):
    k = 2*np.pi*n/L
    omega = c*k
    return np.sin(k * z - omega * t) 

u_true_tz = wave_1D_periodic(times[:,None], z[None,:], L, c, n=1)

fig, axarr = plt.subplots(1, 1, figsize=(7,2), sharex=True, sharey=True)
time_idx = 10
axarr.plot(z, u_true_tz[time_idx])
axarr.set_ylabel("$u(z,t)$")
axarr.set_xlabel("Space $z$ [m]")
axarr.set_title(f"Mode index={n},  t={times[time_idx]:.3f} s")
plt.show()

We can see visualize the travelling in the time with an animation

In [ ]:
from matplotlib.animation import FuncAnimation


# Plot/animate
def animate_wave_1D_periodic_plot(
    u_tx,
    x,
    times,
    title='',
    L=1,
    interval=25,
    x_ref=None,
    u_ref=None,
    show_err=False,
    save_path=None,
    mode='js',
    ):

    fig, ax = plt.subplots(figsize=(7,3))
    n_time, n_space = u_tx.shape
    (line,) = ax.plot(x, u_tx[0], 'C1', lw=2, label='Predicted')

    if u_ref is not None:
        (line_ref,) = ax.plot(x_ref, u_ref[0], 'C0--', lw=2, label='Analytic')
    if u_ref is not None and show_err:
        err = np.abs(u_tx[0]-u_ref[0])
        (line_err,) = ax.plot(x_ref, err, 'k--', lw=1, label='Error')

    # set the plots
    # vmin, vmax = -1.1*np.max(np.abs(u_tx[0])), 1.1*np.max(np.abs(u_tx[0]))
    vmin, vmax = -1.2, 1.2

    ax.set_xlim(0, L)
    ax.set_ylim(vmin, vmax)
    ax.set_xlabel("x [m]")
    ax.set_ylabel("u(x,t)")
    ax.set_title(f"{title}, t=0.000 s")
    ax.legend(loc='upper right')

    def update(i):
        line.set_ydata(u_tx[i])
        if u_ref is not None:
            line_ref.set_ydata(u_ref[i])
        if u_ref is not None and show_err:
         line_err.set_ydata(np.abs(u_tx[i]-u_ref[i]))
        ax.set_title(f"{title}, t={times[i]:.3f} s")
        return (line,)

    anim = FuncAnimation(
        fig, update, frames=n_time, interval=interval,
        blit=True, cache_frame_data=False
    )

    # output: js widget (default), HTML5 video, or GIF
    if mode == "html":
        from IPython.display import HTML
        html = HTML(anim.to_html5_video())   # smoother playback for large Nt
        plt.close(fig)
        if save_path:  # optional save mp4
            try: anim.save(save_path, writer='ffmpeg', fps=max(1,int(1000/interval)))
            except Exception as e: print(f"[warn] save mp4 failed: {e}")
        return html
    elif mode == "gif":
        if save_path is None: save_path = "anim.gif"
        try: anim.save(save_path, writer='pillow', fps=max(1,int(1000/interval)))
        except Exception as e: print(f"[warn] save gif failed: {e}")
        plt.close(fig)
        return HTML(f'<img src="{save_path}">')
    else:
        from matplotlib import rc
        rc('animation', html='jshtml')
        plt.close(fig)   # suppress extra static plot
        return anim




def animate_wave_1D_periodic_scatter(
    u_tx,
    x,
    times,
    title='',
    L=1,
    interval=25,
    x_ref=None,
    u_ref=None,
    show_err=False,
    save_path=None,
    mode='js',
    ):

    fig, ax = plt.subplots(figsize=(7,3))
    n_time, n_space = u_tx.shape
    pts = ax.scatter(x, u_tx[0], s=30, label='Predicted')

    if u_ref is not None:
        (line_ref,) = ax.plot(x_ref, u_ref[0], 'C0--', lw=2, label='Analytic')
    if u_ref is not None and show_err:
        err = np.abs(u_tx[0]-u_ref[0])
        (line_err,) = ax.plot(x_ref, err, 'k--', lw=1, label='Error')

    # set the plots
    # vmin, vmax = -1.1*np.max(np.abs(u_tx[0])), 1.1*np.max(np.abs(u_tx[0]))
    vmin, vmax = -1.2, 1.2

    ax.set_xlim(0, L)
    ax.set_ylim(vmin, vmax)
    ax.set_xlabel("x [m]")
    ax.set_ylabel("U(z,t)")
    ax.set_title(f"{title}, t=0.000 s")
    ax.legend(loc='upper right')

    def update(i):
        pts.set_offsets(np.c_[x, u_tx[i]])
        if u_ref is not None:
            line_ref.set_ydata(u_ref[i])
        if u_ref is not None and show_err:
         line_err.set_ydata(np.abs(u_tx[i]-u_ref[i]))
        ax.set_title(f"{title}, t={times[i]:.3f} s")
        return (pts,)

    anim = FuncAnimation(
        fig, update, frames=n_time, interval=interval,
        blit=True, cache_frame_data=False
    )

    # output: js widget (default), HTML5 video, or GIF
    if mode == "html":
        from IPython.display import HTML
        html = HTML(anim.to_html5_video())   # smoother playback for large Nt
        plt.close(fig)
        if save_path:  # optional save mp4
            try: anim.save(save_path, writer='ffmpeg', fps=max(1,int(1000/interval)))
            except Exception as e: print(f"[warn] save mp4 failed: {e}")
        return html
    elif mode == "gif":
        if save_path is None: save_path = "anim.gif"
        try: anim.save(save_path, writer='pillow', fps=max(1,int(1000/interval)))
        except Exception as e: print(f"[warn] save gif failed: {e}")
        plt.close(fig)
        return HTML(f'<img src="{save_path}">')
    else:
        from matplotlib import rc
        rc('animation', html='jshtml')
        plt.close(fig)   # suppress extra static plot
        return anim



In [ ]:


# already provided function
anim = animate_wave_1D_periodic_plot(
    u_true_tz, z, times, x_ref=z, u_ref=u_true_tz, show_err=True,
    title=f'Periodic traveling wave — Mode index={n}',
    mode='html'
)
anim

# Recap about PINNs

### What:
*PINN* is neural network NN with params $\theta$:
- that learns (encode and decode) a function $g(\mathbf{x}; \theta)$
- enforces the governing physics (e.g., PDEs) directly in the **loss**.


### Why use PINNs?
- **Continuous approach**
    - **No mesh** or grid (vs. CNN) → works on irregular domains
    - compute exact **analytical gradient** of the NN
- Use Neural Networks
  - **Universal function approximators**
  - Can consume any kind of data → extention/condition with **multimodalities**
- Natural **physics + data fusion** (inverse problems)
    - uses PDE as regularization to avoid overfitting / improving generalization
    - can learn unknown PDE parameters (e.g., wave speed c, sources) by backprop.

### PINN ingredients

- **coordinate-based Neural Network** $g(\mathbf{x}; \theta)$ (MLP, SIREN, $\ldots$)
    - must process a single point ($\neq$ CNNs that require mesh / context)
    - this is not "image-to-image" mapping, but coordinate-to-"image"
- **Autodiff** to compute needed derivatives $\nabla g, \nabla^2 g, \ldots $ w.r.t. any (partial) input ${\bf x}$
- **Collocation points** $ \{\mathbf{x}_{PDE}\} \subset \Omega $ to evaluate the PDE residual
- **Initial** and **boundary conditions** $ \{\mathbf{x}_{IC}, \mathbf{x}_{BC}\} \subset \partial\Omega, \: \mathcal{B}({\bf x}_{IC,BC})=0 $ to find unique solution
-  (Optional, maybe noisy) observation (${\bf x}_i,y_i$), so that, $ g(\mathbf{x}_i) \approx y_i $

**General loss function**
$$
\mathcal{L}(\theta)
= \lambda_{\text{PDE}} \!\!\sum_{\mathbf{x}_{PDE}}\!\! \big\|\mathcal{D}[g_\theta](\mathbf{x}_{PDE})\big\|^2
+ \lambda_{\text{BC}} \!\!\sum_{\mathbf{x}_{BC}}\!\! \big\|\mathcal{B}[g_\theta](\mathbf{x}_{BC})\big\|^2
+ \lambda_{\text{IC}} \!\!\sum_{\mathbf{x}_{IC}}\!\! \big\|\mathcal{B}[g_\theta](\mathbf{x}_{IC})\big\|^2
+ \lambda_{\text{data}} \!\!\sum_i\!\! \big\|g({\bf x_i}) - y_i\big\|^2
$$

with $\lambda_{\{\cdot\}}$ being the loss weigth for each term $\{PDE, IC, BC, \text{data}\}$.

## Recall PINNs' Workflow

### Training
```
1) Sample initial/boundary (and data) points.
2) Compute network output (forward pass)
3) Build derivatives via autodiff (backward pass).
4) Compute residuals (and data) loss functions
5) Repeat for all I.C., B.C. (and data)
6) Optimize (Gradient Descent step).  
```

### Test / Evaluation
```
7) Evaluate anywhere (mesh-free inference).
```

# PINN for simulation / forward problems


## PINN for 1D Periodic Traveling Wave (Loop)

### Target PDE
$$
\frac{\partial^2}{\partial z^2} U - \frac{1}{ c^2} \frac{\partial^2}{\partial t^2} U = 0
$$
on $z$ in $[0,L]$ with

> - **boudary conditions**: periodic, that is,
>    - $u(t,0)=u(t,L)$
>    - $u_z(t,0)=u_z(t,L)$
> - **Initial conditions** that produce the pure traveling wave
>    - $u(0,z) =\sin(kz)$ for the initial position
>    - $u_t(0,z) = -\omega\,\cos(kz)$ for the inital velocity

The **analyitical solution** is in the form of
$$U(t,z)=sin(kz - \omega t),$$
with $k=2\pi n/L$, $\omega=c k$ are defined by the BC and IC.

### Architecture

Let's define the architecture, a simple MLP in <a href="https://pytorch.org"><b>PyTorch</b></a>.
<!--
<details>
  <summary>$\mathbb{Q}$: which activation function do we want?</summary>
    $\quad\to$ Inf. differentiable activ. function, e.g., $\tanh$, $\sin$, $\ldots$
</details> -->


In [ ]:
# Plain MLP
class MLP(nn.Module):
    def __init__(self, in_dim=2, hidden=128, depth=4):
        super().__init__()
        act_func = nn.Tanh() ## <- WHICH ONE?
        layers=[]
        for i in range(depth):
            layers += [
                nn.Linear(in_dim if i==0 else hidden, hidden), act_func]
        layers += [nn.Linear(hidden, 1)]
        self.net = nn.Sequential(*layers)
    def forward(self, tx): return self.net(tx)

model = MLP(in_dim=2, hidden=128, depth=3).to(device)

sum_params = sum(p.numel() for p in model.parameters())
print(f"Total num of params: {sum_params/1e3:.1f}k")

### "Training" dataset

We don't have a "dataset" as in classical supervised learning.

Here, we **sample** points and use them to compute the loss.
- **collocation points** in the domain for the evaluating the PDE
- points for the $\ldots$

> - **boudary conditions**: periodic, that is
>    - $U(t,0)=U(t,L)$
>    - $U_z(t,0)=U_z(t,L)$
> - **Initial conditions**, that is
>    - $U(0,z) =\sin(kz)$ for the initial position
>    - $U_t(0,z) = -\,\omega\,\cos(kz)$ for the inital velocity

In this setting, **IC/BC are know analyitical function**, so,
- points can be sampled continuosly and evaluated
- and are provided **as a (Input, label) dataset**.

In [ ]:
# Samplers

# sample point in the domain Omega for the PDE evaluation
def sample_collocation(Nc, L=1, T=1):
    t = torch.rand(Nc,1) * T        # in [0,T]
    z = torch.rand(Nc,1) * L        # in [0,L]
    return torch.cat([t,z], dim=1).to(device)

# sample initial conditions (IC)
def sample_ic(Ni,L=1):
    z = torch.rand(Ni,1) * L      # x in [0,1]
    t = torch.zeros_like(z)
    u0 = torch.sin(k*z)           # <--- HERE
    ut0 = -omega*torch.cos(k*z)   # <--- HERE
    return torch.cat([t,z],1).to(device), u0.to(device), ut0.to(device) # <-- this looks like the classing (input, label) training set

# sample boundary conditions (BC)
def sample_bc(Nb,T=1):
    t = torch.rand(Nb,1) * T      # times in [0,1]
    z0 = torch.zeros_like(t)      # x=0
    zL = torch.ones_like(t)       # x=L=1
    return (torch.cat([t,z0],1).to(device),
            torch.cat([t,zL],1).to(device))

In [ ]:
# A little test
tz_pde = sample_collocation(2048, L=L, T=T)
print('Collocation points', tz_pde.shape)

tz_ic, u_ic, u0_ic = sample_ic(1024, L=L)
print('Initial condition', tz_ic.shape, u_ic.shape, u0_ic.shape)

tz, tzL = sample_bc(256, T=T)
print('Boundary condition', tz.shape, tzL.shape)

### Loss functions and the PDE

We want our NN to satisfy the following equation
$$
\frac{\partial^2}{\partial z^2} U_{\theta} - \frac{1}{ c^2} \frac{\partial^2}{\partial t^2} U_\theta = 0
$$

Which need to be approximated by the following **quadrature** on the sampled collocation points
$$
\mathcal{L}_{PDE} =
    \sum_{{\bf z}_{PDE}} \left(U_{\theta, zz} \bigg\rvert_{z_{PDE}}
        - \frac{1}{c^2} U_{\theta, tt} \bigg\rvert_{z_{PDE}}
    \right)^2
$$

- To compute the above loss function we need $U_{\theta,tt}, U_{\theta,zz}$ of a scalar field $u_\theta(t,z)$, i.e., the neural net's output.

- $U_{tt}, U_{zz}$ are short for the 2nd partial deriv. w.r.t. $t$ and $z$, risp.

We can use `torch.autograd.grad` for this!

#### How to compute derivatives w.r.t. input in Torch

in Pytorch, `torch.autograd.grad` computes the gradient of given tensor **with respect to the input tensor**

```python
torch.autograd.grad(
    outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False,
    ...
)
```

This is different from `torch.autograd.backward` that computes the **sum of the gradients** with respect to the **graph leaves** (usually NN's params).

A note on the use of `autograd.grad`:

- we need to set `requires_grad_(True)` on input tensors
- The `grad_outputs` contains the "vector" in the vector-Jacobian product. To get the correct gradient w.r.t. input we need to pass `torch.ones_like(u)`.
- `create_graph` enables the computation of higher order derivatives (and loss)

In [ ]:
tz = sample_collocation(Nc=5)
tz = tz.requires_grad_(True) # <-- Enable grad here

u = tz ** 2 - tz

u_tz = torch.autograd.grad(u, tz, torch.ones_like(u), create_graph=True)[0] # <-- add args
u_t = u_tz[:,0:1]
u_z = u_tz[:,1:2]

# check with analytical grads
print(torch.allclose(u_t, 2*tz[:,0:1] - 1))
print(torch.allclose(u_z, 2*tz[:,1:2] - 1))

In [ ]:
tz = sample_collocation(Nc=5).requires_grad_(True)

model = MLP(in_dim=2, hidden=128, depth=3).to(device)
u = model(tz) # <-- let s try on our MLP

u_tz = torch.autograd.grad(u, tz, torch.ones_like(u), create_graph=True)[0]
print(u_tz)

### Complexity of the gradient of NN:
- the cost of evaluating the gradient $\partial^n / \partial x^n$ grows exponentially with the order $n$
- the cost of evaluating the first derivatives w.r.t. the the input is about double of the cost of normal training.

## Coding the full PINN

In [ ]:
# Problem parameter
L = const['L'] # domain length
c = const['c'] # wave speed
n = const['n'] # Fourier mode index (1,2,...) <--- try first with n=1
print(f'Loop length: {L:.2f}, wave speed: {c:.2f}, mode index: {n}')

k = 2 * math.pi * n / L
omega = c * k

In [ ]:
# Optimization parameter

N_c = 2048 # num of internal collocation points
N_i = 1024 # num of points for initial condition
N_b = 256  # num of points for boundary condition

# loss weights
w_pde, w_ic, w_bc = 1.0, 1.0, 1.0

# for evaluation purpose
z_eval = torch.linspace(0, L, 400)[:,None]
t_eval = 0.35
tz_eval = torch.cat([torch.full_like(z_eval, t_eval), z_eval], dim=1).to(device)

In [ ]:
# Code a function to get the derivatives from the model's output
def derivatives(u, tz):
    grads = torch.autograd.grad(u, tz, torch.ones_like(u), create_graph=True)[0]
    ut, uz = grads[:, :1], grads[:, 1:]
    utt = torch.autograd.grad(ut, tz, torch.ones_like(ut), create_graph=True)[0][:, :1]
    uzz = torch.autograd.grad(uz, tz, torch.ones_like(uz), create_graph=True)[0][:, 1:]
    return ut, uz, utt, uzz

In [ ]:
# Finally we need the optimization / training part
def train(model, n_iters=500, lr=1e-3, log_every=100, plot_live=False):
    model.train()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    hist = []

    for step in range(n_iters):
        opt.zero_grad(set_to_none=True)

        # (1) sample
        tz_c = sample_collocation(N_c).requires_grad_(True)

        # (2) forward
        u_c = model(tz_c)

        # (3) autodiff + PDE loss
        ut_c, uz_c, utt_c, uzz_c = derivatives(u_c, tz_c)
        r = utt_c - c**2 * uzz_c
        L_pde = torch.mean(r**2)

        # (4) IC losses
        tz_i, u0, ut0 = sample_ic(N_i)
        tz_i = tz_i.requires_grad_(True)
        u_i = model(tz_i)
        ut_i, _, _, _ = derivatives(u_i, tz_i)
        L_ic  = torch.mean((u_i  - u0)**2) + torch.mean((ut_i - ut0)**2)

        # (5) periodic BC losses
        tz0, tzL = sample_bc(N_b)
        tz0 = tz0.requires_grad_(True)
        tzL = tzL.requires_grad_(True)
        u0b = model(tz0)
        uLb = model(tzL)
        _, uz0, _, _ = derivatives(u0b, tz0)
        _, uzL, _, _ = derivatives(uLb, tzL)
        L_bc = torch.mean((u0b - uLb)**2) + torch.mean((uz0 - uzL)**2)

        # (6) total loss & step
        loss = w_pde*L_pde + w_ic*L_ic + w_bc*L_bc
        loss.backward()
        opt.step()

        if step % log_every == 0 or step == n_iters+1:
            parts = {
                'pde' : float(L_pde.detach().cpu()),
                'ic' : float(L_ic.detach().cpu()),
                'bc' : float(L_bc.detach().cpu())
            }
            hist.append((step, float(loss.detach().cpu()), parts))
            print(f"[Adam {step:04d}] total={hist[-1][1]:.3e} parts={parts}")

            if plot_live:
                # optional lightweight live plot
                from IPython.display import clear_output
                clear_output(wait=True)
                steps, totals, comps = zip(*hist)
                pde = [c['pde'] for c in comps]
                ic = [c['ic'] for c in comps]
                bc  = [c['bc']  for c in comps]

                fig, axarr = plt.subplots(1, 3, figsize=(12,3))
                with torch.no_grad():
                    u_pred = model(tz_eval).cpu()
                u_ref  = torch.sin(k*z_eval - omega*t_eval)
                axarr[0].plot(z_eval.squeeze(), u_ref.squeeze(), '--', label='GT', lw=2)
                axarr[0].plot(z_eval.squeeze(), u_pred.squeeze(), label=f'PINN', lw=2)
                axarr[0].legend(loc='upper right')
                axarr[0].set_title(f"Time slice t={t_eval:.2f}")
                axarr[0].set_xlabel("z")
                axarr[0].set_ylabel("U")

                axarr[1].semilogy(steps, totals, label='Total Loss')
                axarr[1].set_title("Total Loss")
                axarr[1].set_xlabel("Training steps")
                axarr[1].grid(True)

                axarr[2].plot(steps, pde, label='PDE')
                axarr[2].plot(steps, ic, label='IC')
                axarr[2].plot(steps, bc, label='BC')
                axarr[2].set_title("Loss partials")
                axarr[2].set_xlabel("Training steps")
                axarr[2].legend(loc='upper right')
                axarr[2].grid(True)

                plt.suptitle(f'Training loop {step+1}/{n_iters}')

                plt.tight_layout()
                plt.show()

    return hist

### PINN training

In [ ]:
reset_seeds(SEED)
model = MLP(in_dim=2, hidden=128, depth=3).to(device)

start = time.time()
_ = train(model, n_iters=300, lr=1e-3, log_every=20, plot_live=True)
print(f"PINN trained! Elapsed time: {time.time()-start:.2f} s")

### Simulate and evaluate on the full domain and animate

In [ ]:
T  = 1.0
Nt = 100
Nz = 200

times  = np.linspace(0, T, Nt)
z = np.linspace(0, L, Nz, endpoint=False)  # periodic grid

grid_tz = np.stack(np.meshgrid(times, z, indexing='ij'), -1)
grid_tz = torch.tensor(grid_tz, dtype=torch.float32, device=device)
# compute analytical solution
u_true_tz = wave_1D_periodic(times[:,None], z[None,:], const['L'], const['c'], const['n'])

# compute PINN predition
with torch.no_grad():
    u_pinn_tz = model(grid_tz).cpu().numpy().squeeze(-1)

animate_wave_1D_periodic_plot(
    u_true_tz, z, times, x_ref=z, u_ref=u_true_tz, show_err=True,
    title=f'Periodic traveling wave — Mode index={n}',
    mode='html'
)

The model has now **learnt the solution** of the PDE given the I.C. and B.C.

Now we can evaluate it at **any point** in $\Omega$

## Perfomance as function of the input dimension, that is, resolution
Lets try to change the number of sampled points, even random scattered points

In [ ]:
Nz_choices = [1, 10, 200, 300, 400, 500, 600, 700, 800, 900, 1000] # <-- trying with diffrect grid resolution
N_runs = 10


# Simulate on the
u_pinn_rntz = []
u_ref_rntz = []
z_grids_rnz = []
timings_n = []
errors_n = []

T  = 1.0
Nt = 100
times  = np.linspace(0, T, Nt)

for Nz in Nz_choices:

    timings_r = []
    errors_r = []


    for r in range(N_runs):
        z = np.sort(np.random.rand(int(Nz)))
        grid_tz = np.stack(np.meshgrid(times, z, indexing='ij'), -1)
        grid_tz = torch.tensor(grid_tz, dtype=torch.float32, device=device)
        u_true_tz = wave_1D_periodic(times[:,None], z[None,:], const['L'], const['c'], const['n'])

        z_grids_rnz.append(z)
        u_ref_rntz.append(u_true_tz)

        start = time.time()
        with torch.no_grad():
          u_pinn_rntz.append(model(grid_tz).cpu().numpy().squeeze(-1))

        timings_r.append(time.time()-start)
        errors_r.append(np.mean((u_pinn_rntz[-1]-u_true_tz)**2))

    timings_n.append(np.mean(timings_r))
    errors_n.append(np.mean(errors_r))

    print(f"Nz={Nz} x Nt={Nt} elapsed time: {timings_n[-1]:.2f} s", f"error: {errors_n[-1]:.2e}")

In [ ]:
fig, axarr = plt.subplots(1,2,figsize=(10,3))
axarr[0].plot(Nz_choices, errors_n)
axarr[0].set_title('Error vs. input size')
axarr[0].set_ylabel('MSE')
axarr[0].set_xlabel("Num of iters")
axarr[1].plot(Nz_choices, timings_n, marker='o')
axarr[1].set_title('Empirical computational time')
axarr[1].set_ylabel('Seconds')
axarr[1].set_xlabel("Num of iters")
plt.show()

## All in one function
For the following steps, it is better to pack the above training code in a single function.

In [ ]:
class PINN_Wave1D(nn.Module):
    def __init__(self,
        model_arch,
        wave_speed=1.0, wave_mode=1, domain_length=1.0,
        samples_sizes = {'N_pde': 2048, 'N_ic' : 1024, 'N_bc' : 256},
        loss_weights = {'pde' : 1., 'ic' : 1., 'bc' : 1.},
        device='cpu',
    ):
        """
        Initialize PINN for wave equation

        Args:
            model_arch: PyTorch model class or instance
            wave_speed: Wave propagation speed (c)
            wave_mode: Wave mode number (k = mode * pi)
            domain_length: Length of spatial domain
            device: Computing device
            N_c, N_i, N_b: Number of collocation, initial, boundary points
        """
        super().__init__()

        self.device = device
        self.c = wave_speed
        self.k = 2 * wave_mode * np.pi / domain_length
        self.omega = self.c * self.k
        self.L = domain_length
        print(f"Wave PINN for c={self.c}, mode={wave_mode}, k={self.k:.3f}, L={self.L:.2f}")

        # Sampling parameters
        self.N_pde = samples_sizes['N_pde']
        self.N_ic = samples_sizes['N_ic'] if 'N_ic' in samples_sizes else 0
        self.N_bc = samples_sizes['N_bc'] if 'N_bc' in samples_sizes else 0

        # Initialize model
        self.model = model_arch.to(device)

        # Loss weights
        self.w_pde = loss_weights['pde']
        self.w_ic = loss_weights['ic'] if 'ic' in loss_weights else 0.
        self.w_bc = loss_weights['bc'] if 'bc' in loss_weights else 0.

        sum_params = sum(p.numel() for p in self.model.parameters())
        print(f"Model params: {sum_params/1e3:.1f}k")

        self.pred_hist = []

    def sample_collocation(self, Nc):
        """Sample collocation points in domain [0,1] x [0,L]"""
        t = torch.rand(Nc, 1)
        x = torch.rand(Nc, 1) * self.L
        return torch.cat([t, x], dim=1).to(self.device)

    def sample_ic(self, Ni):
        """Sample initial condition points"""
        x = torch.rand(Ni, 1) * self.L
        t = torch.zeros_like(x)
        u0 = torch.sin(self.k * x)
        ut0 = -self.omega * torch.cos(self.k * x) # Initial condition on Ut
        return torch.cat([t, x], 1).to(self.device), u0.to(self.device), ut0.to(self.device)

    def sample_bc(self, Nb):
        """Sample boundary condition points for periodic BC"""
        t = torch.rand(Nb, 1)
        x0 = torch.zeros_like(t)
        xL = torch.full_like(t, self.L)
        return (torch.cat([t, x0], 1).to(self.device),
                torch.cat([t, xL], 1).to(self.device))

    def derivatives(self, u, tx):
        """Compute derivatives using autograd"""
        grads = torch.autograd.grad(u, tx, torch.ones_like(u), create_graph=True)[0]
        ut, ux = grads[:, :1], grads[:, 1:]
        utt = torch.autograd.grad(ut, tx, torch.ones_like(ut), create_graph=True)[0][:, :1]
        uxx = torch.autograd.grad(ux, tx, torch.ones_like(ux), create_graph=True)[0][:, 1:]
        return ut, ux, utt, uxx

    def analytical_solution(self, t, x):
        """Compute analytical solution"""
        return torch.sin(self.k * x - self.omega * t)

    def train(self,
            n_iters=1500,
            lr=1e-3,
            log_every=100,
            plot_live=False
        ):
        """Train the PINN model"""
        self.model.train()
        opt = torch.optim.Adam(self.model.parameters(), lr=lr)
        hist = []

        # Evaluation points for live plotting
        if plot_live:
            Nx = 400
            x_eval = torch.linspace(0, self.L, Nx)[:, None]
            t_eval = 0.35
            tx_eval = torch.cat([torch.full_like(x_eval, t_eval), x_eval], dim=1).to(self.device)

        for step in range(n_iters):
            opt.zero_grad(set_to_none=True)

            # (1) PDE loss
            tx_c = self.sample_collocation(self.N_pde).requires_grad_(True)
            u_c = self.model(tx_c)
            ut_c, ux_c, utt_c, uxx_c = self.derivatives(u_c, tx_c)
            r = utt_c - self.c**2 * uxx_c
            L_pde = torch.mean(r**2)

            # (2) IC loss
            tx_i, u0, ut0 = self.sample_ic(self.N_ic)
            tx_i = tx_i.requires_grad_(True)
            u_i = self.model(tx_i)
            ut_i, _, _, _ = self.derivatives(u_i, tx_i)
            L_ic = torch.mean((u_i - u0)**2) + torch.mean((ut_i - ut0)**2)

            # (3) Periodic BC loss
            tx0, txL = self.sample_bc(self.N_bc)
            tx0 = tx0.requires_grad_(True)
            txL = txL.requires_grad_(True)
            u0b = self.model(tx0)
            uLb = self.model(txL)
            _, ux0, _, _ = self.derivatives(u0b, tx0)
            _, uxL, _, _ = self.derivatives(uLb, txL)
            L_bc = torch.mean((u0b - uLb)**2) + torch.mean((ux0 - uxL)**2)

            # (4) Total loss & step
            loss = self.w_pde * L_pde + self.w_ic * L_ic + self.w_bc * L_bc
            loss.backward()
            opt.step()

            if step % log_every == 0:
                parts = {
                    'pde': float(L_pde.detach().cpu()),
                    'ic': float(L_ic.detach().cpu()),
                    'bc': float(L_bc.detach().cpu())
                }
                hist.append((step, float(loss.detach().cpu()), parts))
                print(f"[Adam {step:04d}] total={hist[-1][1]:.3e} parts={parts}")

                if plot_live:
                    self._plot_live_training(hist, tx_eval, x_eval, t_eval, step=step, total_steps=n_iters, title_suffix='')

        self._plot_live_training(hist, tx_eval, x_eval, t_eval, step=step+1, total_steps=n_iters, title_suffix ='')

        u_pred = self.model(tx_eval)
        u_ref = self.analytical_solution(t_eval, x_eval.to(self.device))
        mse = torch.mean((u_pred - u_ref)**2).mean()
        print(f'Training ended. Final MSE {mse.item()}')

        return hist

    def _plot_live_training(self, hist, tx_eval, x_eval, t_eval, step=0, total_steps=0, title_suffix=''):
        """Live plotting during training"""

        try:
            from IPython.display import clear_output
            clear_output(wait=True)
        except ImportError:
            pass  # Skip clearing output if IPython not available

        steps, totals, comps = zip(*hist)
        pde = np.array([c['pde'] for c in comps])
        ic = np.array([c['ic'] for c in comps]) if 'ic' in comps[0] else None
        bc = np.array([c['bc'] for c in comps]) if 'bc' in comps[0] else None
        data = np.array([c['data'] for c in comps]) if 'data' in comps[0] else None

        with torch.no_grad():
            u_pred = self.model(tx_eval).cpu()
        u_ref = self.analytical_solution(t_eval, x_eval.to(self.device)).cpu()

        self.pred_hist.append(
            {'step' : step,
            'x' : x_eval.squeeze(),
            'pred' : u_pred.squeeze(),
            }
        )

        mse = torch.mean((u_pred - u_ref)**2).mean()

        fig, axarr = plt.subplots(1, 4, figsize=(14, 4))
        axarr[0].plot(x_eval.squeeze(), u_ref.squeeze(), '--', label='Analytical True', lw=2)
        axarr[0].plot(x_eval.squeeze(), u_pred.squeeze(), label='PINN', lw=2)
        for r, record in enumerate(self.pred_hist[-5:]):
            axarr[0].plot(record['x'], record['pred'], 'k-', alpha=0.2, lw=2)
        axarr[0].legend(loc='upper right')
        axarr[0].set_title(f"Time t={t_eval:.2f} - MSE {mse:.6f}")
        axarr[0].set_xlabel("z")
        axarr[0].set_ylabel("U")

        axarr[1].semilogy(steps, totals, label='Total Loss')
        axarr[1].set_xlim([0, total_steps])
        axarr[1].set_title("Total Loss")
        axarr[1].set_xlabel("Steps")
        axarr[1].grid(True)

        axarr[2].plot(steps, pde, label='PDE')
        if ic is not None:
            axarr[2].plot(steps, ic, label='IC')
        if bc is not None:
            axarr[2].plot(steps, bc, label='BC')
        if data is not None:
            axarr[2].plot(steps, data, label='Data')
        axarr[2].set_xlim([0, total_steps])
        axarr[2].set_title("Loss Components")
        axarr[2].set_xlabel("Steps")
        axarr[2].legend()
        axarr[2].grid(True)

        axarr[3].plot(steps, self.w_pde * pde, label='PDE')
        if ic is not None:
            axarr[3].plot(steps, self.w_ic * ic, label='IC')
        if bc is not None:
            axarr[3].plot(steps, self.w_bc * bc, label='BC')
        if data is not None:
            axarr[3].plot(steps, self.w_data * data, label='Data')
        axarr[3].set_xlim([0, total_steps])
        axarr[3].set_title("Weigthed Loss Components")
        axarr[3].set_xlabel("Steps")
        axarr[3].legend()
        axarr[3].grid(True)

        plt.suptitle(f'Training loop {step}/{total_steps} iters' + title_suffix)

        plt.tight_layout()
        plt.show()

    def __call__(self, tx):
        return self.model(tx)

Let's train the model!

In [ ]:
reset_seeds(SEED)
mlp_arch = MLP(in_dim=2, hidden=128, depth=3).to(device)
model = PINN_Wave1D(
    mlp_arch,
    wave_speed=const['c'],
    wave_mode=const['n'],
    domain_length=const['L'],
    samples_sizes = {'N_pde': 2048, 'N_ic' : 1024, 'N_bc' : 256},
    loss_weights = {'pde' : 1., 'ic' : 1., 'bc' : 1.},
    device=device,
)

start = time.time()
_ = model.train(n_iters=300, lr=1e-3, log_every=20, plot_live=True)
print(f"Training finished. Elapsed time: {time.time()-start:.2f} s")

We can change different optimization parameters:
- Number of collocation points (or the way we sample them)
- Loss function (e.g., weights)
    - let s try with `BC  = 20`
- Optimizer (e.g., learning rate, n_epochs)
- Architecture (e.g. network, size)

# PINNs for Inverse Problems

If (noisy) observations $\{{\bf x}_i,y_i)\,:\, u(\mathbf{x}_i) \approx y_i\}$ are available, we can add a **data-fit term** to the training loss:
$$
\mathcal J_{\text{data}} = \lambda_{\text{data}} \sum
    (g({\bf x}_i; \theta) - y_i)^2
$$

The final loss is be expandend as
$$
\mathcal J(\theta) = \mathcal J_{\text{PDE}} + \mathcal J_{\text{IC}} + \mathcal J_{\text{BC}} + \mathcal J_{\text{data}}.
$$

This unlock the use of PINNs for **inverse problems**

This adds **data-driven supervision** to the PINN, which can help in several ways:
- **Better converge** (higher accuracy and faster training)
- Enable solving **inverse problems**:
    - Curve fitting / regression / even interpolation (e.g. Sound Field Reconstruction)
    - Learning unknown parameters (e.g., wave speed c) by backprop.
- Replace **unknown B.C. and I.C.** if unknown (which is typical)

From an optimization point-of-view, this is a form of **supervised problem** with **regularization**
$$
\mathcal J(\theta) = \mathcal J_{\text{data}} + \mathcal J_{\text{reg}}
$$
where the **regularization term is the physics (soft) constraint**.

This framework is then related to **regression** (*interpolation*, *curve fitting*, *interpolation*, etc.)
- it's interpolation because the PDE help to resolve unseen signal scales

## Data genetation

In [ ]:
# Let s simulate some noisy data and see how the PINN performs
Nt = 100
Nz = 200
times  = np.linspace(0, T, Nt)
z  = np.linspace(0, L, Nz, endpoint=False)  # periodic grid
u_true_tz = wave_1D_periodic(times[:,None], z[None,:], const['L'], const['c'], const['n'])

reset_seeds(SEED)
N_obs = 80                   # <-- spatial observation !!!

tz_grid = np.stack(np.meshgrid(times, z, indexing='ij'), axis=-1).reshape(-1, 2)
# Randomly sample N_obs spatial points for each time frame
idx = np.random.choice(tz_grid.shape[0], size=N_obs, replace=False)
tz_obs = tz_grid[idx]
# compute analytical solution
u_obs_tz = wave_1D_periodic(tz_obs[:,0], tz_obs[:,1], const['L'], const['c'], const['n'])
# add some noise according to SNR
noise = 0.5 * np.random.randn(*u_obs_tz.shape) # <-- add some noise !!!
u_obs_noisy_tz = u_obs_tz + noise

print('Total values true grid:', tz_grid.shape)
print('Observed locations:', tz_obs.shape)
print('Observed field values:', u_obs_noisy_tz.shape)

For visualization purpose, let's stack all the time frames together

In [ ]:
def plot_wavefield_data(tz_obs, u_obs_tz, u_true_tz):
    vmax = 1.2

    fig, axarr = plt.subplots(1, 2, figsize=(7,3), sharex=True)
    im = axarr[0].imshow(
        u_true_tz, aspect='auto', origin='lower',
        extent=[z.min(), z.max(), times.min(), times.max()],
        vmin=-vmax, vmax=vmax, cmap='viridis'
    )
    axarr[0].set_title('True solution')
    axarr[0].set_xlabel(r'z $\to$')
    axarr[0].set_ylabel(r't $\to$')
    plt.colorbar(im, ax=axarr[0])

    im = axarr[1].imshow(
        u_true_tz, aspect='auto', origin='lower',
        extent=[z.min(), z.max(), times.min(), times.max()],
        vmin=-vmax, vmax=vmax, cmap='viridis'
    )
    axarr[1].scatter(tz_obs[:,1], tz_obs[:,0], c=u_obs_tz, cmap='viridis')
    axarr[1].set_title('Observed points have noise')
    axarr[1].set_xlabel(r'z $\to$')
    axarr[1].set_ylabel(r't $\to$')
    plt.tight_layout()
    plt.show()

plot_wavefield_data(tz_obs, u_obs_noisy_tz, u_true_tz)

## InvPINN Model Class

Let modify the `PINN_Wave1D` class for inverse problems

In practice we **extend the class**, as sampling and derivatives functions remain the same.

In [ ]:
class PINN_Wave1D_inverse(PINN_Wave1D):
    def __init__(self,
        model_arch,
        wave_speed=1.0, wave_mode=1, domain_length=1.0,  # <-- used for viz. purpose
        samples_sizes = {'N_pde': 2048,},
        loss_weights = {'pde' : 1., 'data' : 1.,},
        device='cpu',
        ):
        super().__init__(model_arch, wave_speed, wave_mode, domain_length, samples_sizes, loss_weights, device)
        self.w_pde = loss_weights['pde']
        self.w_data = loss_weights['data'] # <-- need to add this

    def train(self,
              tz_data, u_data,  # <-- need to pass the data in the training function
              n_iters=1500, lr=1e-3, log_every=100, plot_live=False):
        """Train the PINN model"""
        self.model.train()
        opt = torch.optim.Adam(self.model.parameters(), lr=lr)
        hist = []

        # Evaluation points for live plotting
        if plot_live:
            z_eval = torch.linspace(0, self.L, 400)[:, None]
            t_eval = 0.35
            tz_eval = torch.cat([torch.full_like(z_eval, t_eval), z_eval], dim=1).to(self.device)

        for step in range(n_iters):

            opt.zero_grad(set_to_none=True)

            # (1) PDE loss
            tz_c = self.sample_collocation(self.N_pde).requires_grad_(True)
            u_c = self.model(tz_c)
            ut_c, uz_c, utt_c, uzz_c = self.derivatives(u_c, tz_c)
            r = utt_c - self.c**2 * uzz_c
            L_pde = torch.mean(r**2)

            # (4) Data / Supervised losses
            u_d = self.model(tz_data)
            L_data = torch.mean((u_d - u_data)**2)

            # (6) total loss & step
            loss = self.w_pde*L_pde + self.w_data*L_data
            loss.backward()
            opt.step()

            # some loggings
            if step % log_every == 0:
                parts = {
                    'pde': float(L_pde.detach().cpu()),
                    'data': float(L_data.detach().cpu()),
                }
                hist.append((step, float(loss.detach().cpu()), parts))
                print(f"[Adam {step:04d}] total={hist[-1][1]:.3e} parts={parts}")

                if plot_live:
                    self._plot_live_training(hist, tz_eval, z_eval, t_eval, step=step, total_steps=n_iters)

        if plot_live:
            self._plot_live_training(hist, tz_eval, z_eval, t_eval, step=step, total_steps=n_iters)

        u_pred = self.model(tz_eval)
        u_ref = self.analytical_solution(t_eval, z_eval.to(self.device))
        mse = torch.mean((u_pred - u_ref)**2).mean()
        print(f'Training ended. Final MSE {mse.item()}')

        return hist


## Model training

### First a simple MLP

Just set `loss_weights['pde']=0.0`

In [ ]:
tz_data = torch.tensor(tz_obs.reshape(-1,2), dtype=torch.float32, device=device)
u_data = torch.tensor(u_obs_noisy_tz.reshape(-1,1), dtype=torch.float32, device=device)

reset_seeds(SEED)
model_arch = MLP(in_dim=2, hidden=128, depth=4).to(device)
model_inv = PINN_Wave1D_inverse(
    model_arch, wave_speed=1.0, wave_mode=const['n'],
    loss_weights={'pde':0.0, 'data':1.},
    device=device
)

start = time.time()
_ = model_inv.train(tz_data, u_data, n_iters=800, lr=1e-3, log_every=20, plot_live=True)
print(f"MLP training. elapsed time: {time.time()-start:.2f} s")

🧐 Notice:

- While training a smooth nice solution appears, but then we overfit to observations
    - In this lucky case, classical **regularization** would have provided a good fitting
    - e.g., early stopping, L2, less parameters, weight decay, etc.
- We can see tht the **PDE loss is increasing** (evaluated, but not used)
    - -> output is not a "physical field"

And lets visualize the error

In [ ]:
with torch.no_grad():
    Nt = 100
    Nx = 200
    times  = np.linspace(0, T, Nt)
    z  = np.linspace(0, L, Nx, endpoint=False)  # periodic grid
    grid_tz = np.stack(np.meshgrid(times, z, indexing='ij'), -1)
    grid_tz = torch.tensor(grid_tz, dtype=torch.float32, device=device)
    u_mlp_tz = model_inv(grid_tz).cpu().numpy().squeeze(-1)


def plot_wavefield_reconstruction_results(u_pred_tz, u_ref_tz, title='', saveas='img.png'):
    vmax = 1.2
    fig, axarr = plt.subplots(1, 3, figsize=(10,3), sharex=True)
    # Show the true solution as an image
    im = axarr[0].imshow(
        u_ref_tz, aspect='auto', origin='lower',
        extent=[z.min(), z.max(), times.min(), times.max()],
        vmin=-vmax, vmax=vmax, cmap='viridis'
    )
    plt.colorbar(im, ax=axarr[0])
    axarr[0].set_title('True solution')
    axarr[0].set_xlabel(r'x $\to$')
    axarr[0].set_ylabel(r't $\to$')

    im = axarr[1].imshow(
        u_pred_tz, aspect='auto', origin='lower',
        extent=[z.min(), z.max(), times.min(), times.max()],
        vmin=-vmax, vmax=vmax, cmap='viridis'
    )
    plt.colorbar(im, ax=axarr[1])
    axarr[1].set_title('Prediction')
    axarr[1].set_xlabel(r'x $\to$')
    axarr[1].set_ylabel(r't $\to$')

    error = np.abs(u_pred_tz - u_ref_tz)
    mse = np.mean(error**2)
    im = axarr[2].imshow(
        error, aspect='auto', origin='lower',
        extent=[z.min(), z.max(), times.min(), times.max()],
        vmin=-vmax, vmax=vmax, cmap='viridis'
    )
    plt.colorbar(im, ax=axarr[2])
    axarr[2].set_title(f'Error - MSE = {mse:3f}')
    axarr[2].set_xlabel(r'x $\to$')
    axarr[2].set_ylabel(r't $\to$')

    plt.suptitle(title)
    plt.tight_layout()
    plt.savefig(saveas)
    plt.show()

plot_wavefield_reconstruction_results(u_mlp_tz, u_true_tz, title='Model = MLP', saveas='./mlp_reconstuction.png')

### Now the PINN

In [ ]:
reset_seeds(SEED)
model_arch = MLP(in_dim=2, hidden=128, depth=4).to(device)
model_inv = PINN_Wave1D_inverse(
    model_arch, wave_mode=const['n'],
    loss_weights= {'pde': 1., 'data': 1.},
    device=device
)

start = time.time()
_ = model_inv.train(tz_data, u_data, n_iters=800, lr=1e-3, log_every=20, plot_live=True)
print(f"PINN training: elapsed time: {time.time()-start:.2f} s")

And let's appreciate how the PDE terms regularize the training and the resolved  field

### Compare MLP vs PINN

In [ ]:
with torch.no_grad():
    Nt = 100
    Nz = 200
    times  = np.linspace(0, T, Nt)
    z  = np.linspace(0, L, Nz, endpoint=False)  # periodic grid
    grid_tz = np.stack(np.meshgrid(times, z, indexing='ij'), -1)
    grid_tz = torch.tensor(grid_tz, dtype=torch.float32, device=device)
    u_pinn_tz = model_inv(grid_tz).cpu().numpy().squeeze(-1)

plot_wavefield_reconstruction_results(u_pinn_tz, u_true_tz, title='Model = PINN', saveas='./PINN_reconstuction.png')

In [ ]:
plot_wavefield_reconstruction_results(u_mlp_tz, u_true_tz, title='Model = MLP', saveas='./mlp_reconstuction.png')

# PINNs Limitations

## Recap so far

<div style="display: flex; justify-content: space-around;">

<div style="width:45%">
    
**Advantages**
- Mesh-free
- Can solve **forward** and **inverse** problems (even jointly)
- Provide **unsupervised** regularization
- Tractable, analytical gradients
    - e.g., for downstream tasks, or sensitivity analysis

</div>

<div style="width:45%">
    
**Limitations**
- At convergence, **no guardantees** of physics being satisfied
- Can be **hard** to optimize (slow convergence)
- Challenging to scale to **large domain**, **multi-scale** problems
- **Computational cost** due to high-orded gradients

</div>




## Extentions and improvements

- **Architecture**
    - mainly to **overcome the *spectral bias*** (to improve convergence)
        - Random Fourier Features, SIREN, etc.
    - impose **hard constraints**, a.k.a. ansatz, adding *inductive bias*
        - make NN's output that satisfy condition by construction  

- **Optimization**
    - **Loss functions**
        - e.g., adaptive loss tuning / multi-task learning, curriculum learning
    - Sampling **collocation points**
        - e.g., adaptive sampling to drive the optimization where is needed
    - **Optimizer** and scheduling
        - e.g., Adam -> LBFGS; learning-rate warmups/annealing

- **Data / problem** pre-processing and **scalability**
    - **Pre-conditioning**
        - e.g., non-dimensionalization and data normalization
    - **Domain partition** (e.g., deploying multiple PINNs for a Divide et Impera)

- **Theory**
    - Understanding the theoretical limitation of PINNs  
        - **Neural Tangent Kernel Theory**

Reference:
- [[Wang et al, *"An Expert's Guide to Training Physics-informed Neural Networks"*, 2020](https://arxiv.org/abs/2308.08468)]
- [[Moseley et al, *"Finite Basis Physics-Informed Neural Networks (FBPINNs): a scalable domain decomposition approach for solving differential equations"*, 2021](https://arxiv.org/abs/2107.07871)]

## Limitation 1 - PINNs struggle to solve high-frequency / multi-scale problems

![test](https://github.com/Chutlhu/EUSIPCO25_PIML_tutorial/blob/main/images/frequency_bias.png?raw=true)

PINNs suffer from the so-called **Frequency Bias**:
- NN strugges to learn higher frequency (no matter the number of parameters)
- NNs prioritise learning **lower** frequencis function first
- Under some assamption, the **Neural Tangent Kernel theory** can mathematically prove this
![test](https://github.com/Chutlhu/EUSIPCO25_PIML_tutorial/blob/main/images/frequency_bias_matrix.png?raw=true)

References:
- [Rahaman, et al., *"On the Spectral Bias of Neural Networks"*, 2019](https://arxiv.org/abs/1806.08734)
- [Wang, et al., *"When and why PINNs fail to train: A neural tangent kernel perspective"*, 2020](https://arxiv.org/abs/2007.14527)


$\ldots$ luckly some extentions exists

## Better architecture to overcome Spectral Bias

### SIREN: sine-activated MLPs

Proposed bt [Sitzmann et al., *"Implicit Neural Representations with Periodic Activation Functions"*, 2020](https://arxiv.org/abs/2006.09661)

**Why** (vs tanh/ReLU MLPs)
- Battles **spectral bias** → captures high-frequency/detail (e.g., Helmholtz high-k, sharp boundaries)
- Stable gradients over space/time for INRs & PINNs
- Originally proposed for image super-resolution and also PINNs

**How**
For each layer $\ell$
$$
\mathbf{x}_{\ell+1}=\sin\!\big(\omega_0\,\mathbf{W}_\ell \mathbf{x}_\ell+\mathbf{b}_\ell\big)
\quad\text{(final layer often linear)}
$$

![siren](https://github.com/m-pexx/fa25-summer-school/blob/main/images/siren.png?raw=true)

<!-- **Initialization (critical)**
- First layer: $W_0 \sim \mathcal{U}\!\left(-\tfrac{1}{d_{\text{in}}},\tfrac{1}{d_{\text{in}}}\right)$
- Hidden layers: $W_\ell \sim \mathcal{U}\!\left(-\tfrac{\sqrt{6/d_{\text{in}}}}{\omega_0},\tfrac{\sqrt{6/d_{\text{in}}}}{\omega_0}\right)$
- Typical: $\omega_0=30$ for first layer, $\omega_0=1$ for the rest. -->

**When to use**
- Coordinate-based fields (signal/image/audio)
- PDEs with **oscillatory** solutions, multi-scale detail.

**Tips for PINNs**
- Normalize inputs coordinates to $[-1,1]$
- $\omega_0$ should match the "bandwidth" of the signal, i.e., the higher the frequency content, the bigger the $\omega_0$


In [ ]:
# SIREN layer and network (great for oscillatory signals)
class Sine(nn.Module):
    def __init__(self, w0=30.0):
        super().__init__()
        self.w0 = w0
    def forward(self, x):
        return torch.sin(self.w0 * x)

def siren_init(m, w0): # <-- SIREN requires particular initialization
    if isinstance(m, nn.Linear):
        in_f = m.weight.shape[1]
        bound = 1./in_f if hasattr(m, 'is_first') else math.sqrt(6/in_f)/w0
        with torch.no_grad():
            m.weight.uniform_(-bound, bound);
            if m.bias is not None: m.bias.uniform_(-bound, bound)

class SirenNet(nn.Module):
    def __init__(self, in_dim=2, hidden=64, depth=4, w0=30.0, w0_hidden=1.0):
        super().__init__()
        layers=[]
        for i in range(depth):
            lin = nn.Linear(in_dim if i==0 else hidden, hidden)
            if i==0:
                lin.is_first=True
                layers += [lin, Sine(w0)]
            else:
                layers += [lin, Sine(w0_hidden)]
        layers += [nn.Linear(hidden, 1)]

        self.net = nn.Sequential(*layers)
        self.apply(lambda m: siren_init(m, w0_hidden))

    def forward(self, tx): return self.net(tx)

Let s try it on an high-frequency signal

In [ ]:
new_wave_number = 6

# Let s simulate some noisy data and see how the PINN performs
reset_seeds(SEED)
Nt = 100
Nx = 200
times  = np.linspace(0, T, Nt)
x  = np.linspace(0, L, Nx, endpoint=False)  # periodic grid
u_true_tx = wave_1D_periodic(times[:,None], x[None,:],  n=new_wave_number) # <-- new wave number

tx_grid = np.stack(np.meshgrid(times, x, indexing='ij'), axis=-1).reshape(-1, 2)

N_obs = tx_grid.shape[0]
N_obs = 1000 # spatial observation
print(f"Acutally it is {N_obs / tx_grid.shape[0] * 100} %")

# Randomly sample N_obs spatial points for each time frame
idx = np.random.choice(tx_grid.shape[0], size=N_obs, replace=False)
tx_obs = tx_grid[idx]
# compute analytical solution
u_obs_tx = wave_1D_periodic(tx_obs[:,0], tx_obs[:,1], n=new_wave_number) # <-- new wave number
# add some noise according to SNR
noise = 0.1 * np.random.randn(*u_obs_tx.shape)
u_obs_noisy_tx = u_obs_tx + noise

plt.figure(figsize=(7,3))
time_idx = 3                       # choose your frame
t_val = times[time_idx]

# pick the observations at this time
mask = np.isclose(tx_obs[:, 0], t_val)   # safe even with floats
x_obs_t = tx_obs[mask, 1]
u_obs_noisy_t = u_obs_noisy_tx[mask]

# plot line + sparse noisy obs
plt.plot(x, u_true_tx[time_idx], lw=2, label='True')
if x_obs_t.size:
    plt.scatter(x_obs_t, u_obs_noisy_t, s=30, edgecolor='k', label='Noisy obs')
else:
    print(f'No observations at time_idx={time_idx} (t={t_val:.4f}).')
plt.legend(); plt.tight_layout(); plt.show()

print('Total values true grid:', tx_grid.shape)
print('Observed locations:', tx_obs.shape)
print('Observed field values:', u_obs_noisy_tx.shape)

In [ ]:
plot_wavefield_data(tx_obs, u_obs_noisy_tx, u_true_tx)

tx_data = torch.tensor(tx_obs.reshape(-1,2), dtype=torch.float32, device=device)
u_data = torch.tensor(u_obs_noisy_tx.reshape(-1,1), dtype=torch.float32, device=device)

Let s train

### Only SIREN (no PINN)

In [ ]:
new_wave_mode = 6

reset_seeds(SEED)
siren_arch = SirenNet(in_dim=2, hidden=16, depth=3, w0=30.0, w0_hidden=1).to(device) # <-- it is a very small model

sum_params = sum(p.numel() for p in model.parameters())
print(f"Total num of params: {sum_params/1e3:.1f}k")

start = time.time()
model_inv = PINN_Wave1D_inverse(
    siren_arch, wave_mode=new_wave_number,
    loss_weights= {'pde': 0.0, 'data': 1.},
    device=device
)
_ = model_inv.train(tx_data, u_data, n_iters=1000, lr=1e-3, log_every=20, plot_live=True)
print(f"PINN trained. Elapsed time: {time.time()-start:.2f} s")

### PI-SIREN
Adoped in [[Pezzoli et al, 2023](https://dael.euracoustics.org/confs/fa2023/data/articles/001182.pdf), [Karakonstantis et al, 2024](https://arxiv.org/abs/2401.01206)] (more on that later)

In [ ]:
new_wave_mode = 6

reset_seeds(SEED)
siren_arch = SirenNet(in_dim=2, hidden=16, depth=3, w0=30.0, w0_hidden=1).to(device) # <-- it is a very small model

sum_params = sum(p.numel() for p in model.parameters())
print(f"Total num of params: {sum_params/1e3:.1f}k")

start = time.time()
model_inv = PINN_Wave1D_inverse(
    siren_arch, wave_mode=new_wave_number,
    loss_weights= {'pde': 1., 'data': 1.},
    device=device
)
_ = model_inv.train(tx_data, u_data, n_iters=1000, lr=1e-3, log_every=20, plot_live=True)
print(f"PINN trained. Elapsed time: {time.time()-start:.2f} s")

Not working 🤔?

We need to tune the weight for the PDE, e.g. `loss_weights= {'pde': 1e-6, 'data': 1.}`

### We need to tune the weights for the PDE

In [ ]:
new_wave_mode = 6

reset_seeds(SEED)
siren_arch = SirenNet(in_dim=2, hidden=16, depth=3, w0=30.0, w0_hidden=1).to(device) # <-- it is a very small model

sum_params = sum(p.numel() for p in model.parameters())
print(f"Total num of params: {sum_params/1e3:.1f}k")

start = time.time()
model_inv = PINN_Wave1D_inverse(
    siren_arch, wave_mode=new_wave_number,
    loss_weights= {'pde': 1e-6, 'data': 1.}, # <-- change here !!!
    device=device
)
_ = model_inv.train(tx_data, u_data, n_iters=1000, lr=1e-3, log_every=20, plot_live=True)
print(f"PINN trained. Elapsed time: {time.time()-start:.2f} s")

We are getting closer 🙌, but to can we do this **automatically?**

The loss weights may be tuned
- by hand
- grid-search
- using hyper-parameters tuning (e.g. Optuna)

This results in a cumberson tasks, some studies propose to automtically learn them while training (see later)

Or, using **adaptive stragegies**. To know more have a look the EUSIPCO 2025 Tutorial linked at the beginning.

# PINNs for Acoustics — Takeaways

- **Why PINNs?**
    - Mesh-free
    - unify **forward** and **inverse** problems
    - leverage **autodiff** for PDE/BC/IC residuals
    - provide **analytical gradients**
- **Works well when**
    - physics is informative, data are scarce
    - model **continuous** fields (e.g., RIR, HRTF as function of source position).
- **Watch-outs:**
    - spectral bias
    - Loss terms imbalance
    - Optimization hparams, etc.

---


### Practical Recipe (what actually helped)
- **Scaling & nondimensionalization** first (variables & domain)
- **Architectures:** SIREN / positional encodings for oscillatory fields
- **Loss weights:** adaptive weights for balanced training
---

### Diagnostics
- Start with "small" NN
- Evaluated the NN at **off-grid collocation points** (higher scales)
- Investigate if **frequency bias** happens
- Track **PDE vs BC loss** (also **gradient norms** of each loss)
- Manually **tune loss weights**
---

### Caveats (be honest)
- Needs **loss balancing** and **good sampling**
- Higher-order autodiff is costly (+ backprop on it)
- (at this stage) PINNs learn a single solution defined by the BC and IC, not general one.
- Training is sensitive (optimizer/architecture)
- For pure forward sims on simple grids, classic FD/FEM is often faster.

# Thank you very much


## 📚  Further readings (and watching)
- Tutorial: [Physics-informed Machine learning for audio processing @ EUSIPCO 2025](https://github.com/Chutlhu/EUSIPCO25_PIML_tutorial)
- Paper: [An Expert's Guide to Training Physics-informed Neural Networks](https://arxiv.org/abs/2308.08468)
- Paper: [Scientific Machine Learning Through Physics–Informed Neural Networks: Where we are and What’s Next](https://link.springer.com/article/10.1007/s10915-022-01939-z)
- Course: [Dr. Moseley and Prof. Mishra's ETH Course on PINNs](https://youtu.be/D-F7BYRhAkQ?si=Zs6sHTTDTalIC7x0)

## 📚 References

[[1](https://www.sciencedirect.com/science/article/pii/S0021999118307125)] Raissi, Maziar, Paris Perdikaris, and George E. Karniadakis. "Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations." Journal of Computational physics 378 (2019): 686-707.

[[3](https://www.sciencedirect.com/science/article/pii/S095219762030292X)] Nascimento, R. G., Fricke, K., & Viana, F. A. (2020). A tutorial on solving ordinary differential equations using Python and hybrid physics-informed neural network. Engineering Applications of Artificial Intelligence, 96, 103996.

[[4](https://towardsdatascience.com/solving-differential-equations-with-neural-networks-afdcf7b8bcc4)] Dagrada, Dario. "Introduction to Physics-informed Neural Networks" ([code](https://github.com/madagra/basic-pinn)).

[[5](https://towardsdatascience.com/physics-and-artificial-intelligence-introduction-to-physics-informed-neural-networks-24548438f2d5)] Paialunga Piero. "Physics and Artificial Intelligence: Introduction to Physics Informed Neural Networks".
